# $Cross Validation$

We split the training data set into training data and validation data

Ths is done using K-fold Cross Validation

In this method the training data set is split into several groups, with 'k'  number of datas, randomly. From each group 'k-1' number of data is used for training and the remaining '1' data point is used for validation. In such a way 'k-1' data points from all groups are put into the training set and the rest are used for validation set. K is usually set as 10

There is another method Stratified K-fold Cross Validation

This method is used for classification datasets. This methodmakes sure that the validation set has an equal proportion of data from each class tht the data is to be split into

# Implementation

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm

boston = datasets.load_boston()
boston.data.shape, boston.target.shape

D:\jupiter\envs\udemy_ml_python\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing da

((506, 13), (506,))

In [2]:
X_train, X_test, y_train, y_test = train_test_split(boston.data, boston.target, test_size = 0.4, random_state = 0)

print(X_train.shape, y_train.shape)

print(X_test.shape, y_test.shape)

regression = svm.SVR(kernel = 'linear', C = 1).fit(X_train, y_train)
regression.score(X_test, y_test)

(303, 13) (303,)
(203, 13) (203,)


0.6674313821662665

When evaluating different settings (hyperparameters) for estimators, such as the 'C' setting that must be done manually set an SVM. There is still a possibility of overfitting because the parameters can be tweaked until the estimator works ideally

# Computing Cross Validation Metrics

In [3]:
from sklearn.model_selection import cross_val_score
regression = svm.SVR(kernel = 'linear', C = 1)
scores = cross_val_score(regression, boston.data, boston.target, cv = 5)
scores

array([0.77275763, 0.72778244, 0.56131914, 0.1505652 , 0.08212413])

In [4]:
print('Accuracy: %0.2f (+/- %0.2f)' % (scores.mean(),scores.std()))

Accuracy: 0.46 (+/- 0.29)


By default, the score computed at each cross validation iteration is the score method of the estimator. It is possible to change this by using the scoring parameter

In [5]:
from sklearn import metrics
scores = cross_val_score(regression, boston.data, boston.target, cv = 5, scoring = 'neg_mean_squared_error')
scores

array([ -7.8478596 , -24.78180305, -35.13272325, -74.50549945,
       -24.40477437])

# K-Fold

In [6]:
from sklearn.model_selection import KFold

X = ['a', 'b', 'c', 'd']
kf = KFold(n_splits = 2)
for train, test in kf.split(X):
    print('%s, %s' % (train, test))

[2 3], [0 1]
[0 1], [2 3]


# Stratified K Fold

In [7]:
from sklearn.model_selection import StratifiedKFold

X = np.ones(10)
y = [0, 0, 0, 0, 1, 1, 1, 1, 1, 1]
skf = StratifiedKFold(n_splits = 3)
for train, test in skf.split(X, y):
    print('%s %s' % (train, test))

[2 3 6 7 8 9] [0 1 4 5]
[0 1 3 4 5 8 9] [2 6 7]
[0 1 2 4 5 6 7] [3 8 9]


In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA #PCA sclices data in dimensions that helps to explain variation

from sklearn import svm
from sklearn.pipeline import make_pipeline

pipe_svm = make_pipeline(StandardScaler(), 
                         PCA(n_components = 2), 
                         svm.SVR(kernel = 'linear', C = 1))

pipe_svm.fit(X_train, y_train)
y_pred = pipe_svm.predict(X_test)
print('Test Accuracy: %.3f' % pipe_svm.score(X_test, y_test))

Test Accuracy: 0.391


In [9]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(estimator = pipe_svm, 
                         X = X_train,
                         y = y_train, 
                         cv = 10,
                         n_jobs = 1)
print('CV Accuracy scores: %s' % scores)

CV Accuracy scores: [0.63971176 0.43579197 0.46977821 0.25027246 0.5124364  0.26221374
 0.30877195 0.54528563 0.37810066 0.47313549]


In [10]:
print('CV Accuracy: %.3f +/- %.3f' %(np.mean(scores), np.std(scores)))

CV Accuracy: 0.428 +/- 0.121
